In [24]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

import math
import nltk
from nltk.lm.models import Lidstone
from nltk.lm.models import Laplace
from nltk.corpus import machado
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE
import re

from nltk.corpus import machado
import nltk

# Funções

In [46]:
def remover_stopwords(texto):
    # Carregue a lista de stopwords da língua desejada (exemplo: português)
    stopwords_lista = nltk.corpus.stopwords.words('portuguese')

    # Tokenize o texto em palavras
    palavras = word_tokenize(texto)

    # Crie uma lista de palavras sem as stopwords
    palavras_sem_stopwords = [palavra for palavra in palavras if palavra.lower() not in stopwords_lista]

    # Recrie o texto sem as stopwords
    texto_sem_stopwords = ' '.join(palavras_sem_stopwords)

    return texto_sem_stopwords

def generate_ngrams(text, n):
    words = text.split()
    bigrams = []
    for i in range(len(words) - n + 1):
        bigram =  " ".join(words[i:i+n])
        bigrams.append(tuple(bigram.split()))

    return bigrams

def remocao_caracteres_especiais(text):
    padrao = r'[^\w]+'
    return re.sub(padrao, ' ', text)

# Função para gerar texto
def generate_text(seed_text, next_words, model, max_sequence_length):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_length - 1, padding='pre')
        predicted = model.predict(token_list, verbose=0)
        predicted_word_index = np.argmax(predicted)
        predicted_word = tokenizer.index_word[predicted_word_index]
        seed_text += " " + predicted_word
    return seed_text

def calculate_perplexity(model, test_input, tokenizer, max_sequence_length):
    token_list = tokenizer.texts_to_sequences([test_input])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_length - 1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted)
    predicted_word_prob = predicted[0][predicted_word_index]
    perplexity = 1 / predicted_word_prob
    return perplexity

def criar_modelo_n_gram(text,n, model=None, gamma=0.1):
    tokenized_text = [list(map(str.lower, word_tokenize(sent)))
                  for sent in sent_tokenize(text)]
    train_data, padded_sents = padded_everygram_pipeline(n, tokenized_text)

    if model == 'li':
        model_machado = Lidstone(gamma,order=n )
    elif model == 'la':
        model_machado = Laplace(order=n)
    else:
        model_machado = MLE(n)

    model_machado.fit(train_data, padded_sents)
    return model_machado

def trans_corpus_text(corpus):
    text = ''
    for i in corpus:
        text += machado.raw(i) +' '

    #Pre-processamento
    #text = remover_stopwords(text)
    text = remocao_caracteres_especiais(text)

    return text

def model_rnn():
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Embedding(total_words, 64, input_length=max_sequence_length - 1))
  model.add(tf.keras.layers.SimpleRNN(128))
  model.add(tf.keras.layers.Dense(total_words, activation='softmax'))

  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

  return model


In [26]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
nltk.download('machado')

[nltk_data] Downloading package machado to /root/nltk_data...


True

# Corpus

In [5]:
corpus = machado.fileids()

text = ''
cont = 0
total_contos = 1
for i in corpus:
    text += machado.raw(i) +' '
    if cont == total_contos:
        break
    cont+=1

# N Grams

In [64]:
model_n_gram = criar_modelo_n_gram(text,4)

Testando a geração de frases

In [67]:
en = 'O amor que vem'
t = f'{en} '
for i in model_n_gram.generate(20, text_seed=en.split()):
    t += i + ' '
print(t)

O amor que vem cá , adelaide , que era um espírito fraco , cederia ao último que lhe falasse , e os olhos 


Perplexidade infinita

In [71]:
model_n_gram.perplexity('O amor que vem')

inf

# Modelo RNN

In [6]:


# Pré-processamento de texto
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

# Criar sequências de palavras
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Padronizar sequências
max_sequence_length = max([len(x) for x in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

# Dividir em dados de entrada e saída
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

# Model RNN
model = model_rnn()

# Treinamento
model.fit(X, y, epochs=100, verbose=1)


Epoch 1/100
2785/2785 [==============================] - 93s 32ms/step - loss: 6.7921
Epoch 2/100
2785/2785 [==============================] - 75s 27ms/step - loss: 6.0373
Epoch 3/100
2785/2785 [==============================] - 75s 27ms/step - loss: 5.5412
Epoch 4/100
2785/2785 [==============================] - 76s 27ms/step - loss: 5.0991
Epoch 5/100
2785/2785 [==============================] - 76s 27ms/step - loss: 4.6935
Epoch 6/100
2785/2785 [==============================] - 76s 27ms/step - loss: 4.3109
Epoch 7/100
2785/2785 [==============================] - 75s 27ms/step - loss: 3.9573
Epoch 8/100
2785/2785 [==============================] - 74s 27ms/step - loss: 3.6310
Epoch 9/100
2785/2785 [==============================] - 74s 26ms/step - loss: 3.3377
Epoch 10/100
2785/2785 [==============================] - 74s 27ms/step - loss: 3.0865
Epoch 11/100
2785/2785 [==============================] - 75s 27ms/step - loss: 2.8622
Epoch 12/100
2785/2785 [============================

Testando a geração de frases

In [28]:
generated_text = generate_text("O amor que vem", 20, model, max_sequence_length)
print(generated_text)

O amor que vem com este sujeito de quem seria a viúva não sei mas de não é nada menos que ele dizia casar


Perplexidade

In [29]:
test_input = "O amor que vem"
perplexity = calculate_perplexity(model, test_input, tokenizer, max_sequence_length)
print(f"Perplexidade: {perplexity:.2f}")

Perplexidade: 1.28
